In [1]:
import numpy as np
import keras
from keras import backend as k
from keras.models import Sequential
from keras.layers import Activation
from keras.layers.core import Dense,Flatten
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.applications import imagenet_utils
from keras.models import Model
import math
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import *
from matplotlib import pyplot as plt
import itertools
from sklearn.metrics import confusion_matrix
%matplotlib inline

Using TensorFlow backend.


In [2]:
train_path='D:/water_burst/testing/Train'
valid_path='D:/water_burst/testing/Valid'

In [3]:
train_batches=ImageDataGenerator(preprocessing_function=keras.applications.mobilenet.preprocess_input).flow_from_directory(
    train_path,
    target_size=(224,224),
    classes=['Water','Non'],
    batch_size=10
)
valid_batches=ImageDataGenerator(preprocessing_function=keras.applications.mobilenet.preprocess_input).flow_from_directory(
    valid_path,
    target_size=(224,224),
    classes=['Water','Non'],
    batch_size=10
)

Found 3662 images belonging to 2 classes.
Found 24 images belonging to 2 classes.


In [4]:
mobile=keras.applications.mobilenet.MobileNet(weights='D:/water_burst/mobile.h5')

In [5]:
mobile.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32

In [6]:
x=mobile.layers[-6].output
predictions=Dense(2,activation='softmax')(x)
model=Model(inputs=mobile.input,outputs=predictions)

In [7]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128 

In [8]:
for layer in model.layers[:-10]:
    layer.trainable=False

In [9]:
checkpoint = keras.callbacks.ModelCheckpoint("D:/water_burst/mn/weights-{loss:.5f}.h5",
                                             monitor='loss',
                                             verbose=1,
                                             save_best_only=True,
                                             save_weights_only=True,
                                             mode='auto',
                                             period=1)

model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [10]:
model.fit_generator(train_batches,steps_per_epoch=25,validation_data=valid_batches,validation_steps=4,epochs=100,callbacks=[checkpoint])

Epoch 1/100


UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node conv1/convolution (defined at C:\ProgramData\Miniconda3\envs\gpu\lib\site-packages\tensorflow_core\python\framework\ops.py:1751) ]] [Op:__inference_keras_scratch_graph_7475]

Function call stack:
keras_scratch_graph


In [11]:
train_batches.class_indices

{'Water': 0, 'Non': 1}

In [5]:
model.load_weights('D:/test/weights-0.00002.h5')

In [20]:
def prepare_image(file):
    img_path='D:/desk/pic/FreeVideoToJPGConverter/Desktop/test/'
    img=image.load_img(img_path+file,target_size=(224,224))
    img_array=image.img_to_array(img)
    img_array_extended_dims=np.expand_dims(img_array,axis=0)
    return keras.applications.mobilenet.preprocess_input(img_array_extended_dims)

In [24]:
preprocessed_image=prepare_image('VID_20190626_131703 07.jpg')
predict= model.predict(preprocessed_image)
print(predict.argmax())
'''results=imagenet_utils.decode_predictions(predict)
results'''

0


'results=imagenet_utils.decode_predictions(predict)\nresults'

In [27]:
model.save_weights('D:/New folder/experiment4.h5')